## 個人課題1
### Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
* リポジトリ名
* 主要な言語
* スターの数
##### 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する

In [38]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time
import re

In [39]:
# DBファイルの保存先パス
path = ''

# Google Colabの場合
# path = '/content/'

# DBファイル
db_name = 'exercise-1.db'

# DB接続の確立
# DBファイルが存在しない場合は新規作成される
conn = sqlite3.connect(path + db_name)

# DBへの接続を閉じる
conn.close()

### テーブル作成

In [40]:
path = ''
db_name = 'exercise-1.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # すでにテーブルがある場合は作り直すために削除（初期化用）
    cur.execute('DROP TABLE IF EXISTS exercise_1;')

    # SQL文の作成
    # リポジトリ名(repositories_name), 主要な言語(language), スター数(stars)
    sql = 'CREATE TABLE  exercise_1 (id INTEGER PRIMARY KEY AUTOINCREMENT, repositories_name TEXT, language TEXT, stars TEXT);'

    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

### スクレイピング

In [41]:
url = "https://github.com/google"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}  

scraped_data = [] # データベースに挿入するためのリスト

def convert_stars_to_int(stars_text):
    #スター数のテキストを整数に変換する
    if not stars_text or stars_text == "Unknown" or stars_text == "0":
        return 0
    
    # 'k'表記の処理（例: 9.7k -> 9700）
    if 'k' in stars_text.lower():
        return int(float(stars_text.lower().replace('k', '')) * 1000)
    
    # カンマを削除して整数に変換
    return int(stars_text.replace(',', ''))

try:
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    
    #サーバー負荷軽減のため待機
    time.sleep(1) 

    soup = BeautifulSoup(res.text, 'html.parser')
    
    # まず全ての li を取得し、その中にリポジトリ名があるかチェックする
    all_li = soup.find_all('li')
    
    for li in all_li:
        # リポジトリ名 (この属性を持つタグがあるか確認)
        name_tag = li.find('a', itemprop='name codeRepository')
        
        # 名前が見つからない li はリポジトリ情報の行ではないのでスキップ
        if not name_tag:
            continue

        repositories_name = name_tag.text.strip()

        # 言語
        lang_tag = li.find('span', itemprop='programmingLanguage')
        language = lang_tag.text.strip() if lang_tag else "Unknown"

        # スター数
        star_tag = li.find('a', href=re.compile(r'/stargazers$'))
        stars = star_tag.text.strip() if star_tag else "0"
        
        # データをリストに追加
        scraped_data.append((repositories_name, language, stars))
    
    print(f"{len(scraped_data)} 件のデータを取得しました。")

except Exception as e:
    print('スクレイピング中にエラー:', e)


10 件のデータを取得しました。


### データベースに挿入

In [42]:
path = ''
db_name = 'exercise-1.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # 複数レコードの挿入
    # INSERT INTO テーブル名 VALUES (列に対応したプレースホルダーをカンマ区切りで);
    sql = 'INSERT INTO exercise_1 (repositories_name, language, stars) VALUES (?, ?, ?);'


    # SQL文の実行
    cur.executemany(sql, scraped_data)
    
    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

finally:
    # DBへの接続を閉じる
    conn.close()

### データの参照

In [43]:
path = ''
db_name = 'exercise-1.db'

try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)

    # SQL（RDBを操作するための言語）を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # データを参照するSQL
    # SELECT * FROM テーブル名;
    # * の部分は，取得したい列の名前を，区切りで指定することもできる．
    sql = "SELECT * FROM exercise_1;"

    # SQL文の実行
    cur.execute(sql)

    # 変更をDBに反映させる
    conn.commit()

except sqlite3.Error as e:
    print('エラーが発生しました:', e)

else:
    for row in cur:
        # 行データ(row)はタプルなので，アンパックして列データを取得
        id, repositories_name, language, stars = row
        print(f"ID: {id}, リポジトリ名: {repositories_name}, 言語: {language}, スター数: {stars}")

finally:
    # DBへの接続を閉じる
    conn.close()

ID: 1, リポジトリ名: perfetto, 言語: C++, スター数: 4,984
ID: 2, リポジトリ名: nsjail, 言語: C++, スター数: 3,581
ID: 3, リポジトリ名: nomulus, 言語: Java, スター数: 1,767
ID: 4, リポジトリ名: skia, 言語: C++, スター数: 10,266
ID: 5, リポジトリ名: device-infra, 言語: Java, スター数: 58
ID: 6, リポジトリ名: android-cuttlefish, 言語: C++, スター数: 571
ID: 7, リポジトリ名: meridian, 言語: Python, スター数: 1,186
ID: 8, リポジトリ名: angle, 言語: C++, スター数: 3,841
ID: 9, リポジトリ名: heir, 言語: C++, スター数: 608
ID: 10, リポジトリ名: gn-language-server, 言語: Rust, スター数: 18
